# Exploration

In [1]:
## Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import pickle

import torch
import torch.nn as nn

In [2]:
# Load data
path_in = './data/'

train = pd.read_csv(path_in + 'train.csv')
#test = pd.read_csv(path_in + 'test.csv')

Data structure: Question ID (qid), question text, target.

Target 0 - Negative

Target 1 - Positive

Training data:

Around 1,300,000 samples. Will have to be split up into training, validation and testing.

In [3]:
## Train data
print('Number of samples for training: {}\nRatio positives to total: {:.2f}'.format(len(train), 
                                                                                    len(train.loc[train['target'] == 1])/len(train)))

Number of samples for training: 1306122
Ratio positives to total: 0.06


In [3]:
## Example sentence

train['question_text'][8]

'Is there such a thing as dressing moderately, and if so, how is that different than dressing modestly?'

## Embeddings
As a start, the 'glove.840B.300d' embedding is used.

Also, you can use 'torchtext.vocab.GloVe(name='840B', dim=300)' (see [torchtext package](https://torchtext.readthedocs.io/en/latest/vocab.html#vocab) )

In [ ]:
## Read the vectors

path = r'./data/embeddings/'

embeddings_dict = {}

with open(path + 'glove.6B.50d.txt', 'r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:])
        embeddings_dict[word] = vector
        
## Save to disc as pickle
#pickle.dump( embeddings_dict, open( "./data/embeddings/GloVe.p", "wb" ) )

In [4]:
## Load the saved embedding GloVe with dimension 50
embedding = pickle.load(open("./data/embeddings/GloVe50d.p", "rb") )

## Generate input vector for neural net

For each word in dataset’s vocabulary, we check if it is on GloVe’s vocabulary. If it is, we load its pre-trained word vector. Otherwise, we initialize a random vector.

Generate a matrix with dimensions:

(dataset’s vocabulary length, word vectors dimension)

len(target_vocab): Length of the longest sentence? Pad the rest?

In [1]:
# Take sentence as list of words and get weights from embedding.
# Pad with trailing 0s if neccessary

# TODO: Exctract punctuation


def get_weights(sentence):
    ## Split the sentence into words and count them

    print("Sentence:\n" + str(sentence) + "\n")
    
    sentence = sentence.split(' ')
        
    glove = embedding
    emb_dim = 50

    # Find weights<
    matrix_len = len(sentence)
    weights_matrix = np.zeros((matrix_len, 50))
    words_found = 0

    for i, word in enumerate(sentence):
        try: 
            weights_matrix[i] = glove[word]
            words_found += 1
        except KeyError:
            weights_matrix[i] = np.random.normal(scale=0.6, size=(emb_dim, ))

    print("{} / {} words found".format(words_found, len(sentence)))
    
    
    ## Pad with zeros. Longest sentence in data has 44 words. Pad to 45.
    z = np.zeros((1, 50))
    
    for i in range(45-matrix_len):      
        weights_matrix = np.concatenate((weights_matrix, z), axis=0)
         
    
    return weights_matrix

In [2]:
sentence = train['question_text'][10]

weights = get_weights(sentence)

print(weights.shape)

NameError: name 'train' is not defined

## Scale data

(45, 50)

## Neural net

The input dimension for the net are 45 x 50.

Use LSTM

In [19]:
## Net
### Define net for test
class Net(nn.Module):   
    def __init__(self, seq_len, batch_size, hidden_size, input_size, D_out):
        super().__init__()
        ## Embedding layer
        #self.emb = nn.Embedding(output_size, hidden_size)
               
        ## Layer 1
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)      # Add number of layers         
        self.relu1 = nn.ReLU()
        
        ## Output layer
        #n_hidden_1 = hidden_size * 1
        
        #self.fc2 = nn.Linear(n_hidden_1, D_out)
        self.out_act = nn.Sigmoid()
        
            
    def forward(self, x): 
        
        #h0 = (num_layers * num_directions, batch, hidden_size)
        #c0 = (num_layers * num_directions, batch, hidden_size)
        
        ## First layer 
        y = self.lstm1(x)
        
        #y = self.relu1(y)
    
        ## Output layer
        #y = self.out_act(self.fc2(y))
        y = self.out_act(y)
        
        return y

In [20]:
def fit(X, Y, model, criterion, optimizer, epochs=100):
    model.train()
    loss_accuracy = {'training_loss':[], 'validation_accuracy':[]}
    
    for epoch in range(epochs):
        #clear_output(wait=True)
        print("Epoch {} / {}\n=============".format(epoch+1, epochs))
            
        if epoch > 0:
            print("Training loss: {}\nValidation accuracy: {:.2f}%".format(loss.item(), accuracy))
            pass
        
        # Add loop for processing batches
        #for x in X:

        x = X
        y = Y

        optimizer.zero_grad()
                ## Forward pass
            #print(x.shape)

        yhat = model(x)

        print(yhat.shape)
                ## Compute loss
        loss = criterion(yhat, y)
                ## Compute gradient in backward pass
        loss.backward()
                ## Update weights
        optimizer.step() 

        loss_accuracy['training_loss'].append(loss.item())
             
    print("Training complete!")
                
    return loss_accuracy

In [21]:
epochs = 2
learning_rate = 0.1

seq_len = 45
batch_size = 45
input_size = 50
hidden_size = 1
D_out = 1

data = torch.from_numpy(weights)

## Load data
#train_loader = torch.utils.data.DataLoader(dataset=data, batch_size=45, shuffle=True)
#validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset), shuffle=False)

#data = data.transpose(0,1)

data = data.view(-1, 45, 50)
data = data.float()

y = torch.tensor(1).view(-1,1)

## Initialize model
net = Net(seq_len, batch_size, hidden_size, input_size, D_out)
#net.apply(weights_init)
print(net.parameters)

optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
criterion = nn.BCELoss()

## Train the model
loss_accuracy = fit(data, y, net, criterion, optimizer, epochs=epochs)

<bound method Module.parameters of Net(
  (lstm1): LSTM(50, 1, batch_first=True)
  (relu1): ReLU()
  (out_act): Sigmoid()
)>
Epoch 1 / 2


TypeError: sigmoid(): argument 'input' (position 1) must be Tensor, not tuple

In [7]:

data = torch.from_numpy(weights)
#data = data.transpose(0,1)
data = data.view(-1, 50, 45)
#data = data.float()

data.shape

torch.Size([1, 50, 45])

In [8]:
data

tensor([[[ 1.0403, -0.0799,  0.8967,  ..., -0.6373,  1.2048,  0.6531],
         [ 0.4826,  0.0020, -0.6749,  ...,  0.1296,  0.2725, -0.3295],
         [ 0.3421,  0.1561,  0.2095,  ..., -0.8044,  0.1839, -0.3433],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
       dtype=torch.float64)

In [29]:
def create_emb_layer(weights_matrix, non_trainable=False):
    num_embeddings, embedding_dim = weights_matrix.size()
    emb_layer = nn.Embedding(num_embeddings, embedding_dim)
    emb_layer.load_state_dict({'weight': weights_matrix})
    if non_trainable:
        emb_layer.weight.requires_grad = False

    return emb_layer, num_embeddings, embedding_dim

class ToyNN(nn.Module):
    def __init__(self, weights_matrix, hidden_size, num_layers):
        super(self).__init__()
        self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(embedding_dim, hidden_size, num_layers, batch_first=True)
        
    def forward(self, inp, hidden):
        return self.gru(self.embedding(inp), hidden)
    
    def init_hidden(self, batch_size):
        return Variable(torch.zeros(self.num_layers, batch_size, self.hidden_size))